In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_excel('./国网业务详细指标.xlsx', sheet_name = 0)
data_dict = pd.read_excel('./data.xlsx', sheet_name = 0)

In [3]:
data_dict['key'] = data_dict['key'].apply(lambda x: x.strip())

In [4]:
data_dict = dict(zip(data_dict['key'],data_dict['value']))

In [5]:
def trans(x):
    trans_x = 0
    if re.search('Free disk space on [/a-z0-9]+ \(percentage\)', x):
        if re.search('/[a-z0-9]+', x):
            trans_x = '{}目录下磁盘空间空闲百分比'.format(re.search('/[a-z0-9]+', x).group())
        else:
            trans_x = '/目录下磁盘空间空闲百分比'
    elif re.search('Used disk space on [/a-z0-9]+', x):
        if re.search('/[a-z0-9]+', x):
            trans_x = '{}目录下磁盘空间已使用字节数'.format(re.search('/[a-z0-9]+', x).group())
        else:
            trans_x = '/目录下磁盘空间空闲百分比'
    elif re.search('Free inodes on [/a-z0-9]+ \(percentage\)', x):
        if re.search('/[a-z0-9]+', x):
            trans_x = '{}目录下inodes空间空闲百分比'.format(re.search('/[a-z0-9]+', x).group())
        else:
            trans_x = '/目录下磁盘空间空闲百分比'
    elif re.search('Free disk space on [/a-z0-9]+', x):
        if re.search('/[a-z0-9]+', x):
            trans_x = '{}目录下磁盘空间空闲字节数'.format(re.search('/[a-z0-9]+', x).group())
        else:
            trans_x = '/目录下磁盘空间空闲百分比'
    elif re.search('Total disk space on [/a-z0-9]+', x):
        if re.search('/[a-z0-9]+', x):
            trans_x = '{}目录下磁盘总空间字节数'.format(re.search('/[a-z0-9]+', x).group())
        else:
            trans_x = '/目录下磁盘总空间字节数'
    elif re.search('Incoming network traffic on [a-z0-9]+', x):
        trans_x = '网卡{}: 进口流量'.format(re.search('[a-z0-9-]+$', x).group())
    elif re.search('Outgoing network traffic on [a-z0-9]+', x):
        trans_x = '网卡{}: 出口流量'.format(re.search('[a-z0-9-]+$', x).group())
    elif re.search('ceph-droped-in[-a-z0-9]+', x):
        new = re.search('ceph-droped-[in0-9]+-', x).group()
        trans_x = '网卡{}: 进口丢包数'.format(x.split(new)[1])
    elif re.search('Incoming network dropped packets on [-a-z0-9]+', x):
        new = re.search('Incoming network dropped packets on ', x).group()
        trans_x = '网卡{}: 进口丢包数'.format(x.split(new)[1])
    elif re.search('[A-Za-z ]+ process is running', x):
        new = re.search(' process is running', x).group()
        trans_x = '{}进程正在运行'.format(x.split(new)[0])
    elif re.search('[A-Za-z ]+ is listening on port', x):
        new = re.search(' is listening on port', x).group()
        trans_x = '{}进程正在被侦听'.format(x.split(new)[0])
    elif re.search('OpscDB storage Assigned volume [{}a-z0-9-]+', x):
        new = re.search('OpscDB storage Assigned volume ', x).group()
        trans_x = '{}分配存储卷'.format(x.split(new)[1])
    elif x in list(data_dict.keys()):
        for i in list(data_dict.keys()):
            if i == x:
                trans_x = data_dict[i]
    return trans_x

In [6]:
data = data[['indexName']].dropna()

In [7]:
data['itemname'] = data['indexName'].apply(lambda x: trans(x.strip()))

In [8]:
data.to_excel('data1.xlsx', sheet_name = 'APM')

In [6]:
xl = pd.ExcelFile('./国网业务详细指标.xlsx').sheet_names
write = pd.ExcelWriter('./data1.xlsx')
for i in xl:
    print(i)
    data = pd.read_excel('./国网业务详细指标.xlsx', sheet_name = i)
    data = data[['indexName']].dropna()
    data['itemname'] = data['indexName'].apply(lambda x: trans(x.strip()))
#     print(data.iloc[0])
    data.to_excel(write, sheet_name = i)
write.save()
write.close()

APM
Ceph,Discovered hosts
Discovered hosts,K8s
Openstack，MySQL
Openstack Cluster
Mysql_Server,Openstack Cluster
MySQL
Zabbixserver


### 修改关于mysql的内容

In [39]:
data = pd.read_excel('./国网业务详细指标.xlsx', sheet_name = 'Zabbixserver')

In [40]:
data = data[['indexName']].iloc[83:].dropna()

In [41]:
data_dict = pd.read_excel('./data.xlsx', sheet_name = 2)
data_dict['key'] = data_dict['key'].apply(lambda x: x.strip())
data_dict = dict(zip(data_dict['key'],data_dict['value']))

In [42]:
def trans_mysql(x):
    trans_x = 0
    if x in list(data_dict.keys()):
        for i in list(data_dict.keys()):
            if i == x:
                trans_x = data_dict[i]
    return trans_x

In [43]:
data['itemname'] = data['indexName'].apply(lambda x: trans(x.strip()))

In [44]:
data.to_excel('./data1.xlsx')

### 添加Application

In [60]:
def application(x):
    application = 0
    if re.search('Free disk space on [/a-z0-9]+ \(percentage\)', x):
        application = 'Filesystems'
    elif re.search('Used disk space on [/a-z0-9]+', x):
        application = 'Filesystems'
    elif re.search('Free inodes on [/a-z0-9]+ \(percentage\)', x):
        application = 'Filesystems'
    elif re.search('Free disk space on [/a-z0-9]+', x):
        application = 'Filesystems'
    elif re.search('Total disk space on [/a-z0-9]+', x):
        application = 'Filesystems'
    elif re.search('Incoming network traffic on [a-z0-9]+', x):
        application = 'Network'
    elif re.search('Outgoing network traffic on [a-z0-9]+', x):
        application = 'Network'
    elif re.search('ceph-droped-in[-a-z0-9]+', x):
        application = 'Network'
    elif re.search('Incoming network dropped packets on [-a-z0-9]+', x):
        application = 'Network'
    elif re.search('[A-Za-z ]+ process is running', x):
        application = 'Process'
    elif re.search('[A-Za-z ]+ is listening on port', x):
        application = 'Service'
    elif re.search('OpscDB storage Assigned volume [{}a-z0-9-]+', x):
        application = 'Volume'
    elif x in list(data_dict.keys()):
        for i in list(data_dict.keys()):
            if i == x:
                application = data_dict[i]
    return application

In [61]:
data_dict = pd.read_excel('./data.xlsx', sheet_name = 0)
data_dict['key'] = data_dict['key'].apply(lambda x: x.strip())
data_dict = dict(zip(data_dict['key'],data_dict['application']))

In [62]:
xl = pd.ExcelFile('./国网业务详细指标.xlsx').sheet_names
write = pd.ExcelWriter('./data1.xlsx')
for i in xl:
    print(i)
    data = pd.read_excel('./国网业务详细指标.xlsx', sheet_name = i)
    data = data[['indexName']].dropna()
    data['application'] = data['indexName'].apply(lambda x: application(x.strip()))
#     print(data.iloc[0])
    data.to_excel(write, sheet_name = i)
write.save()
write.close()

APM
Ceph,Discovered hosts
Discovered hosts,K8s
Openstack，MySQL
Openstack Cluster
Mysql_Server,Openstack Cluster
MySQL
Zabbixserver
